In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib notebook
plt.rcParams["figure.figsize"] = (14,7)

In [ ]:
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve

# Essai Euler implicite
E=2
delta=1 
T=50
u = lambda t: (1-np.cos(2*np.pi*t/T))*np.cos(E*t+ np.sin(np.pi *t/T)/(np.pi/T))
psi_0=np.array([0,0,-1])
dt=1e-3
Omega_x=np.array([[0,0,0],[0,0,-1],[0,1,0]])
Omega_z=np.array([[0,-1,0],[1,0,0],[0,0,0]])

def euler_explicit(psi_0,t_0,t_f,E,delta,dt):
    n = int((t_f - t_0) / dt) 
    psi=np.zeros((n+1, 3))
    psi[0]=psi_0
    i=0
    for t_j in np.arange (t_0, t_f, dt):
        psi[i+1]= psi[i] + dt *np.dot((E*Omega_z+delta*u(t_j)*Omega_x),psi[i])
        i+=1
    return psi

def euler_implicit_itératif(psi_0,t_0,t_f,E,delta,dt):
    n = int((t_f - t_0) / dt) 
    psi=np.zeros((n+1, 3))
    psi[0]=psi_0
    Omega_z_constant = E * Omega_z # Pré_calcul
    i=0
    for t_j in np.arange (t_0, t_f, dt):
        Omega_x_u_tj = delta * u(t_j)  * Omega_x  # Pré-calcul
        p_psi=psi[i]
        for _ in range(1000):
            new_psi= p_psi + dt *np.dot((Omega_z_constant+Omega_x_u_tj),p_psi)
            if np.linalg.norm(new_psi - p_psi) < 1e-2:
                psi[i+1]=new_psi
                break
            p_psi=new_psi
        i+=1
    return psi


def euler_implicit_fsolve(psi_0,t_0,t_f,E,delta,dt):
    n = int((t_f - t_0) / dt) 
    psi=np.zeros((n+1, 3))
    psi[0]=psi_0
    i=0
    for t_j in np.arange (t_0, t_f, dt):
        fonction_eq = lambda Z : Z - psi[i] - dt *np.dot((E*Omega_z + delta*u(t_j) * Omega_x),psi[i])
        psi[i+1] = fsolve(fonction_eq, psi[i])
        i+=1
    return psi

print (euler_implicit_itératif(psi_0,0,T,E, delta,dt))

# print(euler_implicit_fsolve(psi_0,0,T,E,delta,dt))

# Tracer la trajectoire du spin sur la sphère de Bloch (en 3D)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

Y_vals_ex= euler_explicit(psi_0,0,T,E, delta,dt)
# coordonnées x, y, z de chaque psi(t) sur la sphère
x_vals_ex = Y_vals_ex[:, 0]
y_vals_ex= Y_vals_ex[:, 1]
z_vals_ex= Y_vals_ex[:, 2]

# trajectoire dans l'espace
#ax.plot(x_vals_ex, y_vals_ex, z_vals_ex, label='Trajectoire de $\psi(t)$ avec explicit', color='purple')

Y_vals= euler_implicit_itératif(psi_0,0,T,E, delta,dt)
# coordonnées x, y, z de chaque psi(t) sur la sphère
x_vals = Y_vals[:, 0]
y_vals = Y_vals[:, 1]
z_vals = Y_vals[:, 2]

# trajectoire dans l'espace
ax.plot(x_vals, y_vals, z_vals, label='Trajectoire de $\psi(t)$', color='blue')

Y_vals_fsolve= euler_implicit_fsolve(psi_0,0,T,E, delta,dt)
# coordonnées x, y, z de chaque psi(t) sur la sphère
x_vals_fsolve = Y_vals_fsolve[:, 0]
y_vals_fsolve = Y_vals_fsolve[:, 1]
z_vals_fsolve = Y_vals_fsolve[:, 2]

# trajectoire dans l'espace
#ax.plot(x_vals_fsolve, y_vals_fsolve, z_vals_fsolve, label='Trajectoire de $\psi(t)$ avec fsolve', color='red')

# sphère de rayon 1
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))

# Tracer de la sphère de rayon 1
ax.plot_surface(x, y, z, color='r', alpha=0.1, rstride=5, cstride=5, antialiased=True)